### Imports

In [2]:
import numpy as np
import pandas as pd

import os

from keras.preprocessing.text import Tokenizer
# from keras.utils.np_utils import to_categorical
# from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

# from keras.layers.embeddings import Embedding
from keras.layers import Embedding
from keras.initializers import Constant
from keras.layers import Input

from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Dense
from keras.models import Model
from keras.layers import Bidirectional, GRU, BatchNormalization

from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef

In [ ]:
reuters = pd.read_pickle('reuters_news_concatenated.pkl', 'bz2')

In [ ]:
reuters

### Vectorize News into a 2D integer Tensor

In [ ]:
MAX_SEQUENCE_LENGTH = 512 # 定义输入序列的最大长度，这里假设文本序列的最大长度为512个词
MAX_NUM_WORDS = 20000 # 定义词典的最大大小，即词典中允许出现的不同词的数量，这里假设词典最大大小为20000

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS) # 创建一个Tokenizer对象，并指定最大词数为MAX_NUM_WORDS。Tokenizer用于将文本序列转换为整数序列
tokenizer.fit_on_texts(reuters.news) # 将训练数据reuters.news传入Tokenizer中，用于构建词典，并计算词频等信息

reuters_sequences = tokenizer.texts_to_sequences(reuters.news) # 将文本数据reuters.news转换为整数序列，每个词用其在词典中的索引表示

word_index = tokenizer.word_index # 获取词典的字典表示，其中键为单词，值为单词在词典中的索引
reuters_sequences = pad_sequences(reuters_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post') 
# 对整数序列进行填充（padding）操作，使所有序列长度统一为MAX_SEQUENCE_LENGTH。padding='post'表示在序列末尾进行填充
# 如果序列长度小于MAX_SEQUENCE_LENGTH，则用0进行填充

In [ ]:
len(word_index) # 输出词典的大小，即词典中不同词的数量

In [ ]:
reuters_sequences # 输出填充后的整数序列，即用整数表示的文本数据，所有序列长度都为MAX_SEQUENCE_LENGTH

### Cluster Labels

0 - Sharp Fall			
1 - Modest Fall
2 - Modest Rise
3 - Sharp Rise

In [ ]:
clusters = 4 # 定义了聚类数量，即希望将reuters.Y分为4个不同的类别
labels = np.copy(reuters.Y) # 创建一个名为labels的数组，将reuters.Y的值复制到这个新数组中。这是为了不影响原始数据，以便进行后续的聚类
for i in range(1, clusters):
    print(np.percentile(reuters.Y, 100*i/clusters)) 
# 计算了在当前聚类分割下，reuters.Y中的百分之多少的数据位于该分割值以下。换句话说，它输出了按照百分位数划分后，每个聚类的分割值

In [ ]:
clusters = 4
labels = np.copy(reuters.Y)
labels[reuters.Y<np.percentile(reuters.Y, 100/clusters)] = 0 # 将reuters.Y中小于第一个聚类分割值的数据，标记为类别0
for i in range(1, clusters):
    labels[reuters.Y>np.percentile(reuters.Y, 100*i/clusters)] = i # 将reuters.Y中大于第i个聚类分割值的数据，标记为类别i
reuters.Y = labels.astype("int") # 将更新后的labels数组转换为整型，并将其赋值给reuters.Y，从而实现了数据的离散化

In [ ]:
unique, counts = np.unique(reuters.Y, return_counts=True) # 计算reuters.Y数组中不重复的类别（unique）和每个类别出现的次数（counts）
print(np.asarray((unique, counts)).T) # 使得每一行表示一个类别和它对应的出现次数并打印

In [ ]:
reuters_labels = to_categorical(reuters.Y) 
# reuters.Y是原始的类别标签数组，通过to_categorical函数转换为reuters_labels矩阵
# 每个整数类别标签被转换成了一个长度为类别数的二进制向量，其中只有对应类别的位置上为1，其余位置上都为0

In [ ]:
reuters_labels

### Train/Test Split

In [ ]:
print('Shape of news tensor:', reuters_sequences.shape) # 二维矩阵的形状，该矩阵包含了所有新闻文本的向量化表示。输出的形状是一个元组，表示矩阵的行数和列数。这里输出的形状表示有多少个新闻文本和每个新闻文本的向量长度
print('Shape of label tensor:', reuters_labels.shape) # 这个矩阵包含了所有新闻文本的标签向量。输出的形状是一个元组，表示矩阵的行数和列数。这里输出的形状表示有多少个新闻文本和每个新闻文本的标签向量长度
# print('Shape of binary label tensor:', reuters_bin_labels.shape)

In [ ]:
mask_train = np.random.rand(len(reuters)) < 0.7  # 70% for training
mask_val = (np.random.rand(len(reuters)) >= 0.7) & (np.random.rand(len(reuters)) < 0.8)  # 10% for validation
mask_test = np.random.rand(len(reuters)) >= 0.8  # 20% for testing

train_X = reuters_sequences[mask_train]  # Training set news text vectors
train_Y = reuters_labels[mask_train]  # Training set labels

val_X = reuters_sequences[mask_val]  # Validation set news text vectors
val_Y = reuters_labels[mask_val]  # Validation set labels

test_X = reuters_sequences[mask_test]  # Testing set news text vectors
test_Y = reuters_labels[mask_test]  # Testing set labels

### Prepare Embedding Matrix

In [ ]:
import urllib.request
if (not os.path.isfile('glove.42B.300d.zip') and
   not os.path.isfile('glove.42B.300d.txt')):
    urllib.request.urlretrieve('http://nlp.stanford.edu/data/glove.42B.300d.zip', 
                              os.path.join(os.getcwd(), 'glove.42B.300d.zip'))

import zipfile
if not os.path.isfile('glove.42B.300d.txt'):
    with zipfile.ZipFile("glove.42B.300d.zip","r") as zip_ref:
        zip_ref.extractall(os.getcwd())

glove_wordmap = {} # 创建一个空字典glove_wordmap，用于存储每个单词及其对应的词向量
with open('glove.42B.300d.txt', "r", encoding='utf8') as glove:
    for line in glove:
        word, vector = tuple(line.split(" ", 1)) # 创建一个空字典glove_wordmap，用于存储每个单词及其对应的词向量
        glove_wordmap[word] = np.fromstring(vector, sep=" ") # 单词作为键，对应词向量（通过将字符串转换为NumPy数组）作为值，存到glove_wordmap字典

# def sentence2sequence(tokens):
#     global glove_wordmap
   
#     feature = np.zeros([0, 300])
#     for token in tokens:
#         try:
#             feature = np.vstack((feature, glove_wordmap[token]))
#         except:
#             pass
   
#     return feature

In [ ]:
# embedding_matrix = np.zeros((len(word_index), 300))
embedding_matrix = np.zeros((MAX_NUM_WORDS, 300)) # 创建一个全零二维数组embedding_matrix，300是GloVe词向量的维度
count=0 # 初始化变量count为0，用于计算成功映射到GloVe词向量的单词数量
for word, i in word_index.items(): # 循环遍历词汇表word_index中的每个单词及其对应的索引i
    word_vector = glove_wordmap.get(word) # 从预训练的GloVe词向量字典glove_wordmap中尝试获取当前单词word对应的词向量word_vector
    if word_vector is not None:
        # words not found in embedding index will be all-zeros.
        try:
            embedding_matrix[i] = word_vector # 将word_vector赋值给embedding_matrix的第i行，即将当前单词对应GloVe词向量存储到embedding_matrix
        except:
            pass # GloVe词向量中没有当前单词的对应向量，则跳过
        count=count+1 # 表示成功映射一个单词的词向量

In [ ]:
count/len(word_index) # 给出成功映射到GloVe词向量的单词在词汇表中所占比例

In [ ]:
embedding_matrix

In [ ]:
# load pre-trained word embeddings into an Embedding layer 一个Keras嵌入层的实例，用于将文本序列中的整数标记转换为密集向量的词嵌入
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(MAX_NUM_WORDS,
                            300,
                            embeddings_initializer=Constant(embedding_matrix), # 设置嵌入矩阵的初始值为embedding_matrix，该矩阵包含了预训练的词嵌入向量
                            input_length=MAX_SEQUENCE_LENGTH, #  输入序列的长度，即文本中每个序列的单词数。这个值必须与之前数据预处理中填充（padding）后的长度相同
                            trainable=False) # 设置为False，冻结嵌入层的权重，这样在训练过程中不会更新这些预训练的词嵌入向量
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32') # 创建一个Keras输入层，指定输入数据的形状，每个输入序列是一个长度为MAX_SEQUENCE_LENGTH的整数数组
embedded_sequences = embedding_layer(sequence_input) # 将输入数据序列传递给嵌入层，进行词嵌入的转换。输出embedded_sequences将是一个三维张量，形状为(batch_size, MAX_SEQUENCE_LENGTH, 300)，其中batch_size表示一次训练中输入的样本数量
# 通过这样的操作，可以在模型中使用预训练的词嵌入来表示文本序列，同时保持这些词嵌入不可训练（固定）

### Neural Architectue (1D convnet with global maxpooling)

In [ ]:
def CNN_1D_GMP(clusters=4):
    model = Conv1D(128, 5, activation='relu')(embedded_sequences) # 这是一个1D卷积层，128是输出的特征图数量（filters），5是卷积核大小，激活函数使用ReLU
    model = MaxPooling1D(5)(model) # 这是一个1D最大池化层，用于对卷积层的输出进行降采样。5是池化窗口大小，表示在每5个特征图中选取最大值，将特征图的大小减小
#     model = Dropout(0.2)(model)
    model = Conv1D(128, 5, activation='relu')(model) # 又是一个1D卷积层，和第1层类似，但这次的输入是上一层的输出结果
    model = MaxPooling1D(5)(model) # 同样是一个1D最大池化层，将前面卷积层的输出再次进行降采样
    model = Dropout(0.2)(model) # 这是一个Dropout层，用于在训练过程中随机断开一部分神经元的连接，以防止过拟合。0.2表示断开神经元连接的概率为20%
    model = Conv1D(128, 5, activation='relu')(model) # 1D卷积层，同样的参数设置
    model = GlobalMaxPooling1D()(model) # 这是一个全局最大池化层，对最后一个卷积层的输出进行池化。它在所有特征图上选择最大的值，将所有特征图降维为一个具有最高特征值的向量
#     model = Dropout(0.2)(model)
    model = Dense(128, activation='relu')(model) # 这是一个全连接层（Dense层）用于对全局最大池化层的输出进行处理，有128个神经元，使用ReLU激活函数

    model = Model(sequence_input, Dense(clusters, activation='softmax')(model)) # 使用Keras函数式API将模型的输入和输出连接起来，构建一个最终的模型
    # sequence_input是之前定义的Keras输入层，Dense(clusters, activation='softmax')是一个全连接层，有clusters个神经元，使用softmax激活函数。这个层用于将前面处理的特征映射到clusters个类别上，生成预测的概率分布
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc']) # 编译模型，指定损失函数为交叉熵（categorical_crossentropy），优化器为RMSprop，评估指标为准确率（acc）
    return model
# 该函数定义了一个包含多个1D卷积层和全连接层的模型，适用于文本分类等任务；最后输出层使用softmax激活函数以获取每个输入文本序列属于不同类别的概率

### Neural Architectue (Bidirectional GRU)

In [ ]:
def BI_DIR_GRU(clusters=4):
    model = Bidirectional(GRU(128, return_sequences=True, activation='relu'))(embedded_sequences) # 这是一个双向GRU层，128是GRU单元的数量，return_sequences=True表示返回完整的序列输出而不是最后一个时间步的输出，activation='relu'是激活函数ReLU
    # embedded_sequences是之前定义的嵌入层的输出，作为输入序列
    model = Bidirectional(GRU(128, return_sequences=True, activation='relu'))(model) # 又是一个双向GRU层，输入是前一层的输出
    model = BatchNormalization(axis=-1)(model) # 这是一个批量归一化层，用于规范化输入数据，有助于加速训练过程和提高模型的稳定性
    model = Flatten()(model) # 这是一个展平层，将多维输入展平为一维，为后面的全连接层做准备
    model = Dense(128,activation='relu')(model) # 这是一个全连接层，有128个神经元，使用ReLU激活函数
    model = BatchNormalization(axis=-1)(model) # 又是一个批量归一化层，对全连接层的输出进行规范化
#     model = Dropout(0.2)(model)
#     model = Dense(128,activation='relu')(model)
#     model = BatchNormalization(axis=-1)(model)

    model = Model(sequence_input, Dense(clusters, activation='softmax')(model)) # 同样使用Keras函数式API将模型的输入和输出连接起来，构建最终的模型
    # sequence_input是输入层，Dense(clusters, activation='softmax')是输出层，有clusters个神经元，使用softmax激活函数

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc']) # 编译模型，指定损失函数为交叉熵（categorical_crossentropy），优化器为RMSprop，评估指标为准确率（acc）
    return model

### Train CNN_1D_GMP

In [ ]:
model = CNN_1D_GMP(clusters=4)
# model = load_model('CNN_1D_GMP.h5')
model.fit(train_X, train_Y, # train_X 是训练数据的特征（输入），train_Y 是对应的标签（输出）
          batch_size=128, # 每个批次的样本数量为128
          epochs=10, 
          validation_data=(val_X, val_Y)) 
model.save('CNN_1D_GMP.h5') # 将训练好的模型保存到名为 CNN_1D_GMP.h5 的文件中。这样，训练后的模型可以在以后进行加载和使用，而无需重新训练

### Test CNN_1D_GMP

In [ ]:
predictions = np.argmax(model.predict(test_X), axis=-1) # 从模型对测试集每个样本的预测概率分布中选取最大概率所对应的类别索引，即选取了模型认为最有可能的类别
conf = confusion_matrix(np.argmax(test_Y, axis=-1), predictions) # 计算了混淆矩阵，用于评估模型的分类性能
# np.argmax(val_Y, axis=-1) 是将验证集的真实标签（one-hot 编码的标签）转换为类别索引，以便与模型的预测结果进行比较
# confusion_matrix函数接受真实标签和预测标签作为输入，然后计算并返回一个混淆矩阵，矩阵的每个元素 (i, j) 表示真实类别为i，但模型预测为类别j的样本数量

In [ ]:
predictions

In [ ]:
np.argmax(test_Y, axis=-1)

In [ ]:
pd.DataFrame(conf,
             index = [i for i in ['Sharp Fall', 'Modest Fall', 'Modest Rise', 'Sharp Rise'] ],
             columns = [i for i in ['Sharp Fall', 'Modest Fall', 'Modest Rise', 'Sharp Rise']])

In [ ]:
pd.DataFrame(conf,
             index = [i for i in ['Strong Sell', 'Sell', 'Buy', 'Strong Buy'] ],
             columns = [i for i in ['Strong Sell', 'Sell', 'Buy', 'Strong Buy']])

In [ ]:
conf = np.array(conf)

precision = []
recall = []
f1_score = []

for i in range(4):
    tp = conf[i, i]
    fp = sum(conf[:, i]) - tp
    fn = sum(conf[i, :]) - tp

    p = tp / (tp + fp) if (tp + fp) != 0 else 0
    r = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1 = 2 * p * r / (p + r) if (p + r) != 0 else 0

    precision.append(p)
    recall.append(r)
    f1_score.append(f1)
    print("Label %d Precision: %.2f%% Recall: %.2f%% F1-Score: %.2f%%" % (i, p * 100, r * 100, f1 * 100))


In [ ]:
macro_f1 = sum(f1_score) / len(f1_score)
print("Macro F1-Score: %.2f%%" % (macro_f1 * 100))

In [ ]:
matthews_corrcoef(np.argmax(test_Y, axis=-1), predictions) # 高于0.2的值表示模型性能相对较好，高于0.4表示性能很好，高于0.6表示性能非常好
# 一种用于衡量二分类或多分类模型性能的评估指标，它考虑了真阳性、假阳性、真阴性和假阴性之间的关系，适用于不平衡数据集。值越高，模型的性能越好

### Train BI_DIR_GRU

In [ ]:
model = BI_DIR_GRU(clusters=4)
# model = load_model('BI_DIR_GRU.h5')
model.fit(train_X, train_Y,
          batch_size=128,
          epochs=10,
          validation_data=(val_X, val_Y))
model.save('BI_DIR_GRU.h5')

### Evaluate BI_DIR_GRU

In [ ]:
predictions = np.argmax(model.predict(test_X), axis=-1)
conf = confusion_matrix(np.argmax(test_Y, axis=-1), predictions)

In [ ]:
predictions

In [ ]:
np.argmax(test_Y, axis=-1)

In [ ]:
pd.DataFrame(conf,
             index = [i for i in ['Sharp Fall', 'Modest Fall', 'Modest Rise', 'Sharp Rise'] ],
             columns = [i for i in ['Sharp Fall', 'Modest Fall', 'Modest Rise', 'Sharp Rise']])

In [ ]:
pd.DataFrame(conf,
             index = [i for i in ['Strong Sell', 'Sell', 'Buy', 'Strong Buy'] ],
             columns = [i for i in ['Strong Sell', 'Sell', 'Buy', 'Strong Buy']])

In [ ]:
conf = np.array(conf)

precision = []
recall = []
f1_score = []

for i in range(4):
    tp = conf[i, i]
    fp = sum(conf[:, i]) - tp
    fn = sum(conf[i, :]) - tp

    p = tp / (tp + fp) if (tp + fp) != 0 else 0
    r = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1 = 2 * p * r / (p + r) if (p + r) != 0 else 0

    precision.append(p)
    recall.append(r)
    f1_score.append(f1)
    print("Label %d Precision: %.2f%% Recall: %.2f%% F1-Score: %.2f%%" % (i, p * 100, r * 100, f1 * 100))

In [ ]:
matthews_corrcoef(np.argmax(test_Y, axis=-1), predictions) 